## CNN的基本模板（面向过程）

In [ ]:
# coding: utf-8
import tensorflow as tf

## 基本卷基层的数据结构

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


# x为训练图像的占位符、y_为训练图像标签的占位符

# 一般定义输入层数据和输出层数据用占位符
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])



# 将单张图片从784维向量重新还原为28x28的矩阵图片，这个的意思就是输入的图片要是二维的结构
x_image = tf.reshape(x, [-1, 28, 28, 1])



# 定义一层卷积层
# 定义权重，满足正态分布，且要有一定的深度(32)
W_conv1 = weight_variable([5, 5, 1, 32])
# 以为结构和权重的深度是一样的维度32
b_conv1 = bias_variable([32])
# 进行卷积运算，实际上是一元回归套一个relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化作用，其实是一个选择最大特征的过程，维度是２＊２可以自己定义
 h_pool1 = max_pool_2x2(h_conv1)

  


    
# 全连接层，输出为1024维的向量
# 第一层的全连接层，实质是最后一层卷积池化后的扁平化数据
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
# 依然需要线性变化和激活函数
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# 使用Dropout，keep_prob是一个占位符，训练时为0.5，测试时为1
 keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# 把1024维的向量转换成10维，对应10个类别，对应的生成一个10维的数据
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
#　经过深度网络的最终目的是将原始的数据经过一些列的线性和为线性变化，把数据变成和标签一样维度的数据然后用梯度下降拟合
 y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


    
    
 # 定义模型的梯度 并以一定的学习率进行计算梯度 ，这里的步骤基本上是死的！
# 我们不采用先Softmax再计算交叉熵的方法，而是直接用tf.nn.softmax_cross_entropy_with_logits直接计算
cross_entropy = tf.reduce_mean(
     tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
# 同样定义train_step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)





# 训练20000步
for i in range(20000):
     batch = mnist.train.next_batch(50)
    # 每100步报告一次在验证集上的准确度
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    # sess的核心用法，随即梯度batch的循环的跑模型
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   

#


# 定义测试的准确率
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#这是在测试集测试模型的精准度，其实也是一个死的过程。
print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


## CNN基本模板（面向对象版本）